<div class="alert alert-block alert-info" style="margin-top: 20px">

      
| Name | Description | Date
| :- |-------------: | :-:
|Reza Hashemi| Recurrent Neural Networks - 8th  | Finalized on 23rd of August 2019 | width="750" align="center"></a></p>
</div>

# Recurrent Neural Networks
- IMDB review sentiment classification with RNN
  - Last time, we have started sentence classification with CNN and have achieved accuracy over 0.80. 
  - This time, we try training with RNN to model text data

In [1]:
!pip3 install torch torchvision

In [2]:
import numpy as np
import pandas as pd
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.1.0'

## 1. Import & process dataset
- IMDB review dataset for sentiment analysis
  - [source](http://ai.stanford.edu/~amaas/data/sentiment/)
  - Let's cheat a while and use dataset provided by Keras

In [12]:
from keras.datasets import imdb
from keras.preprocessing import sequence

num_words = 10000
maxlen = 50

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

X_train = sequence.pad_sequences(X_train, maxlen = maxlen, padding = 'pre')
X_test = sequence.pad_sequences(X_test, maxlen = maxlen, padding = 'pre')
    
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(25000, 50) (25000, 50) (25000,) (25000,)


## 2. Creating RNN model and training

- Create and train RNN model for sentence classification, with one GRU layer


![](http://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Gated_Recurrent_Unit%2C_base_type.svg/440px-Gated_Recurrent_Unit%2C_base_type.svg.png)

In [0]:
class imdbTrainDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.X = X_train
    self.y = y_train
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
  
  def __len__(self):
    return len(self.X)
  
class imdbTestDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.X = X_test
    self.y = y_test
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
  
  def __len__(self):
    return len(self.X)

In [0]:
# create dataset & dataloader instances
train_dataset = imdbTrainDataset()
test_dataset = imdbTestDataset()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 128, shuffle = False)

In [0]:
# create RNN with one GRU layer
class net(nn.Module):
  def __init__(self, input_dim, num_words, embedding_dim, hidden_size, device):
    super(net, self).__init__()
    self.input_dim = input_dim
    self.embedding_dim = embedding_dim
    self.hidden_size = hidden_size
    self.device = device
           
    self.embedding = nn.Embedding(num_words, self.embedding_dim)
    
    # recurrent layer (GRU)
    self.rnn = nn.GRU(input_size = self.embedding_dim, hidden_size = hidden_size)
    self.dense = nn.Linear(hidden_size, 2)     
    
  def forward(self, x):

    x = self.embedding(x)                                   # project to word embedding space
    
    h0 = torch.from_numpy(np.zeros((1, x.size(1), self.hidden_size))).float().to(self.device)
    x, _ = self.rnn(x, h0)
    x = x[:, -1, :]
    x = self.dense(x)
    return x

In [0]:
# hyperparameters
DEVICE = torch.device('cuda')
INPUT_DIM = maxlen
EMBEDDING_DIM = 50
HIDDEN_SIZE = 10
NUM_WORDS = num_words
LEARNING_RATE = 1e-3
NUM_EPOCHS = 30        

In [0]:
model = net(INPUT_DIM, NUM_WORDS, EMBEDDING_DIM, HIDDEN_SIZE, DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()   # do not need softmax layer when using CEloss criterion
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [20]:
# training for NUM_EPOCHS
for i in range(NUM_EPOCHS):
  temp_loss = []
  for (x, y) in train_loader:
    x, y = x.long().to(DEVICE), y.to(DEVICE)  # beware that input to embedding should be type 'long'
    outputs = model(x)
    loss = criterion(outputs, y)
    temp_loss.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print("Loss at {}th epoch: {}".format(i, np.mean(temp_loss)))

Loss at 0th epoch: 0.6952009976518397
Loss at 1th epoch: 0.6851094559747346
Loss at 2th epoch: 0.6772835418886068
Loss at 3th epoch: 0.6664662525361899
Loss at 4th epoch: 0.6530864472900119
Loss at 5th epoch: 0.6424068513573432
Loss at 6th epoch: 0.6320125168683578
Loss at 7th epoch: 0.6241002338273185
Loss at 8th epoch: 0.6168175138989274
Loss at 9th epoch: 0.6094661896326103
Loss at 10th epoch: 0.6023486341748919
Loss at 11th epoch: 0.5983551295132054
Loss at 12th epoch: 0.5934095917915811
Loss at 13th epoch: 0.5865354537963867
Loss at 14th epoch: 0.5853558701215958
Loss at 15th epoch: 0.5806262949291541
Loss at 16th epoch: 0.5780014173716915
Loss at 17th epoch: 0.5758882417362563
Loss at 18th epoch: 0.5726898300404452
Loss at 19th epoch: 0.5701178166331077
Loss at 20th epoch: 0.5684995956870974
Loss at 21th epoch: 0.5667622991058291
Loss at 22th epoch: 0.565681440793738
Loss at 23th epoch: 0.5648493728771502
Loss at 24th epoch: 0.5619799300115935
Loss at 25th epoch: 0.55988847996507

## 3. Evaluation
- Evaluate the trained RNN model with accuracy score 
  - Store probability of each instance to a list and compare it with true y label

In [21]:
y_pred, y_true = [], []
with torch.no_grad():
  for x, y in test_loader:
    x, y = x.long().to(DEVICE), y.to(DEVICE)       # beware that input to embedding should be type 'long'
    outputs = F.softmax(model(x)).max(1)[-1]       # predicted label
    y_true += list(y.cpu().numpy())                # true label
    y_pred += list(outputs.cpu().numpy())   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [22]:
# evaluation result
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.59796